# Combining AIS data

AIS data record ship location information; we have acquired two years of ship location information from Spire Inc.

The two years of data are stored on the GOST team's S3 bucket, but we will look at other opportunities to store the data, probably on DDH.

In [1]:
import sys, os, json, time, multiprocessing
import rasterio, boto3, pygeohash, pyarrow

import geopandas as gpd
import pandas as pd
import pyarrow.feather as feather

from shapely.geometry import Point, LineString

sys.path.append("../../gostrocks/src/")

import GOSTRocks.rasterMisc as rMisc

def tPrint(s):
    print("%s\t%s" % (time.strftime("%H:%M:%S"), s))

In [2]:
aws_bucket = "wbgdecinternal-ntl"
path = "AIS"
out_folder = "/home/wb411133/data/Global/AIS"
docs_folder = "../references"
ship_types_file = os.path.join(docs_folder, 'ship_types.json')
with open(ship_types_file, 'r') as ship_file: # https://faq.spire.com/determining-ais-ship-type
    ship_types = json.load(ship_file)

ship_status_file = os.path.join(docs_folder, 'ship_statuses.json')
with open(ship_status_file, 'r') as ship_status: 
    ship_status = json.load(ship_status)

In [ ]:
ship_status

In [ ]:
ship_types

In [3]:
# List all the AIS files on the S3 bucket
client = boto3.client('s3')
ais_file_list = client.list_objects_v2(Bucket=aws_bucket, Prefix='AIS', MaxKeys=5000)

keep_processing = True
continuation_token = ''
try:
    del final
except:
    pass
loop_cnt = 0

# Generate a list of all the files, using the continuation_token 
while keep_processing:
    loop_cnt = loop_cnt + 1    
    try:
        ais_file_list = client.list_objects_v2(Bucket=aws_bucket, Prefix='AIS', ContinuationToken=continuation_token)
    except:
        ais_file_list = client.list_objects_v2(Bucket=aws_bucket, Prefix='AIS')
    keep_processing = ais_file_list['IsTruncated']
    try:
        continuation_token = ais_file_list['NextContinuationToken']
    except:
        pass
    try:
        final = final + ais_file_list['Contents']
    except:
        final = ais_file_list['Contents']

In [4]:
final[0]

{'Key': 'AIS/WorldBank_SAIS_globalAOI_20190101_20201231_000000000000.csv',
 'LastModified': datetime.datetime(2021, 6, 23, 13, 25, 35, tzinfo=tzutc()),
 'ETag': '"223aeec2df6a341bcefbd1dce9997f0c-37"',
 'Size': 303449488,
 'StorageClass': 'STANDARD'}

In [ ]:
def read_process(path):
    good_cols = ['timestamp','mmsi','status','ship_and_cargo_type','latitude','longitude']
    res = pd.read_csv(path)
    tPrint(os.path.basename(path))
    return(res.loc[:,good_cols])

def driver_func(file_list):
    n_processes = round(multiprocessing.cpu_count() * 0.8)
    with multiprocessing.Pool(n_processes) as pool:
        res = pool.map(read_process, file_list)
    return(res)

In [ ]:
startIdx = 0
step = 10
for endIdx in range(step, len(final) + 1, step):
    sel_files = []
    for ais_file in final:
        file_idx = int(ais_file['Key'].split("_")[-1][:-4])
        if file_idx >= startIdx and file_idx < endIdx:
            sel_files.append(os.path.join("s3://", aws_bucket, ais_file['Key']))
    res = driver_func(sel_files)
    try:
        del all_data
    except:
        pass

    for r in res:
        try:
            all_data = all_data.append(r)
        except:
            all_data = r
    break
    feather.write_feather(all_data, os.path.join(out_folder, f"AIS_Combined_2019_2020_{startIdx}_{endIdx}.feather"))
    startIdx = endIdx
    

In [ ]:
ship_status

# DEBUGGING

In [ ]:
inData = read_process(sel_files[0])

In [ ]:
geom = [Point(x['longitude'], x['latitude']) for idx, x in inData.iterrows()]
all_data_geom = gpd.GeoDataFrame(inData, geometry=geom, crs="epsg:4326")
all_data_geom.to_file(os.path.join(out_folder, "AIS_data_0.shp"))

In [ ]:
# The geohash bit is cool but didn't shrink size at all
# curD['geohash'] = curD.apply(lambda x: pygeohash.encode(x['latitude'], x['longitude']), axis=1)

In [ ]:

feather.write_feather(curD.loc[:,['timestamp','mmsi','status','ship_and_cargo_type','latitude','longitude']], 
                      os.path.join(out_folder, ais_file_info['Key'].replace(".csv", ".feather")))

In [ ]:
feather.write_feather(curD, 
                      os.path.join(out_folder, ais_file_info['Key'].replace(".csv", "_full.feather")))

In [ ]:
feather.write_feather(curD.loc[:,['timestamp','geohash']], 
                      os.path.join(out_folder, ais_file_info['Key'].replace(".csv", "_small.feather")))

In [ ]:
curD.columns

# Rasterize results

In [ ]:
# rasterize data based on ship type and month.
global_pop = "/home/public/Data/GLOBAL/Population/WorldPop_PPP_2020/ppp_2020_1km_Aggregated.tif"
inD = all_data_geom
inD['yr_month'] = inD['timestamp'].apply(lambda x: x[:7])

In [ ]:
def generate_raster(xD, templateRaster):
    return(rMisc.rasterizeDataFrame(xD, '', templateRaster=templateRaster, mergeAlg='ADD'))

def write_rasters(res, out_folder, prefix='C'):
    for idx, row in res.iteritems():
        out_file = os.path.join(out_folder, f'{prefix}_{idx}.tif')
        if not os.path.exists(out_file):
            profile = row['meta'].copy()            
            data = row['vals']
            #data[data < 0] = 0
            #data = data.astype('byte')
            profile.update(dtype=data.dtype, compress='lzw')
            #profile['compress'] = 'JPEG'
            with rasterio.open(out_file, 'w', **profile) as outR:
                outR.write_band(1, data)
            del profile
                

In [ ]:
inD_grouped = inD.groupby(["yr_month"])
inD_rasters = inD_grouped.apply(lambda x: generate_raster(x, global_pop))

In [ ]:
write_rasters(inD_rasters[:1], out_folder)

# Generating vector paths

In [ ]:
inD.loc[inD['mmsi'] == 231356000].sort_values(['timestamp'])

In [ ]:
LineString(inD.loc[inD['mmsi'] == 231356000,'geometry'].values)

In [ ]:
selD = inD.loc[~inD['latitude'].isna()]
inD_mmsi = selD.groupby('mmsi')
all_vals = []
for idx, row in inD_mmsi:
    if row.shape[0] > 1:
        row = row.sort_values('timestamp')
        shp = LineString(row.loc[:,'geometry'].values)                      
        all_vals.append([idx, shp])

In [ ]:
row.longitude.min()

In [ ]:
row.longitude.max()

In [ ]:
lineD = pd.DataFrame(all_vals, columns=['ID','geometry'])
#lineD['geometry'] = lineD['geometry'].apply(lambda x: x.buffer(0))
linD_geom = gpd.GeoDataFrame(lineD, geometry='geometry', crs='epsg:4326')

In [ ]:
linD_geom.to_file(os.path.join(out_folder, "AIS_Routes.shp"))

In [ ]:
linD_geom.head()

In [5]:
misc

NameError: name 'misc' is not defined

In [6]:
import GOSTRocks.misc as misc

In [133]:
inShp = "/home/public/Data/COUNTRY/BDI/training_areas_plus_KPG.shp"
inD = gpd.read_file(inShp)
raster_file = "/home/public/Data/COUNTRY/BDI/North/IMG_PHR1B_MS_002/TEST/IMG_PHR1B_MS_201602260834109_ORT_69f9d6e3-e511-4d36-cd41-cd7487f13a10-002_R1C1__BD1_BK4_SC4-8_TRdmp-fourier-gabor-grad-hog-lac-lbpm-lsr-mean-orb-pantex-saliency-seg-sfs-evi.vrt"
inR = rasterio.open(raster_file)
inD = inD.to_crs(f'epsg:{inR.crs.to_epsg()}')

In [110]:
inD2 = misc.explodeGDF(inD)


In [25]:
inD2.to_file('/home/wb411133/temp/training_areas_plus_KPG.shp')

In [111]:
out_points = '/home/wb411133/temp/training_areas_plus_KPG_points.shp'

In [85]:
import importlib

from math import ceil
from shapely.geometry import Point, Polygon

In [117]:
importlib.reload(misc)
try: 
    del final
except:
    pass
idField = 'ClassID'

for idx, row in inD2.iterrows():
    b = row['geometry'].bounds
    res = misc.createFishnet(b[0],b[2],b[1],b[3],inR.res[0],inR.res[0],'POINT',inR.crs.to_epsg(),'')
    resIdx = res.intersects(row['geometry'])
    res = res.loc[resIdx]
    res['ID'] = row[idField]
    try:
        final = final.append(res)
    except:
        final = res

In [118]:
final.to_file(out_points)

In [137]:
from shapely.geometry import box
final_idx = final.intersects(box(*inR.bounds))

In [139]:
final_intersect = final.loc[final_idx]
final_intersect.to_file(out_points)

In [ ]:
inR = rasterio.open()

In [125]:
xx = inR.read(1)

In [126]:
xx.shape

(1399, 1223)

In [129]:
xx.min()

0.0

In [140]:
s_file = '/home/public/Data/COUNTRY/BDI/North/spfeas/training/training_areas_plus_KPG_points__IMG_PHR1B_MS_201602260834109_ORT_69f9d6e3-e511-4d36-cd41-cd7487f13a10-002_R1C1__BD1_BK4_SC8-16_TRmean_SAMPLES.txt'
xx = pd.read_csv(s_file)
xx.head()

,Id,X,Y,IMG_PHR1B_MS_201602260834109_ORT_69f9d6e3-e511-4d36-cd41-cd7487f13a10-002_R1C1__BD1_BK4_SC8-16_TRmean.1,IMG_PHR1B_MS_201602260834109_ORT_69f9d6e3-e511-4d36-cd41-cd7487f13a10-002_R1C1__BD1_BK4_SC8-16_TRmean.2,IMG_PHR1B_MS_201602260834109_ORT_69f9d6e3-e511-4d36-cd41-cd7487f13a10-002_R1C1__BD1_BK4_SC8-16_TRmean.3,IMG_PHR1B_MS_201602260834109_ORT_69f9d6e3-e511-4d36-cd41-cd7487f13a10-002_R1C1__BD1_BK4_SC8-16_TRmean.4,response
0,0.0,29.353466,-3.392174,9.2413,281.4936,5.3620,515.3516,3.0
1,1.0,29.353540,-3.392174,8.7982,282.7966,5.2390,506.9937,3.0
2,2.0,29.353615,-3.392174,8.5744,247.7385,5.0478,496.9581,3.0
3,3.0,29.353687,-3.392174,8.5860,233.9749,4.6859,391.9447,3.0
4,4.0,29.353762,-3.392174,8.7685,252.9411,4.6205,350.3257,3.0


In [141]:
xx['response'].value_counts()

7.0     16573
3.0      4746
0.0      4708
6.0      3560
2.0      1853
8.0       782
10.0      365
1.0        39
Name: response, dtype: int64

In [143]:
inD2.groupby(['ClassID']).first()

,id,Class,Location,classvalue,geometry
ClassID,,,,,
0,0,informal,sororezo,0,"POLYGON ((29.40098 -3.38828, 29.40118 -3.38828..."
1,0,low income,kanyosha,1,"POLYGON ((29.36000 -3.43610, 29.36028 -3.43600..."
2,0,middle income,kibenga,2,"POLYGON ((29.34249 -3.41715, 29.34340 -3.41741..."
3,0,high income,rohero,3,"POLYGON ((29.35328 -3.39219, 29.35379 -3.39214..."
4,0,commercial,rohero,4,"POLYGON ((29.36207 -3.38177, 29.36269 -3.38204..."
5,0,industrial,rohero,5,"POLYGON ((29.35010 -3.37597, 29.35114 -3.37576..."
6,0,Agriculture,None,6,"POLYGON ((29.36934 -3.43715, 29.36930 -3.43685..."
7,0,OpenWater,None,7,"POLYGON ((29.32382 -3.43749, 29.32519 -3.43466..."
8,0,RiverWater,None,8,"POLYGON ((29.34033 -3.43187, 29.34036 -3.43180..."
